In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import pickle
import math
from itertools import product 
from IPython.display import display
#import brightway2 as bw

### read in TRL (ready for two extreme scn SSP119 and SSP585) for each tech under v_year[t] 

In [2]:
partial_TRL = pd.read_excel("../data/tech_list/techlist.xlsx", sheet_name = "gtruck")  
partial_TRL.head()

,Module,Tech,LCIA_perunit,TRL_SSP585_2030,TRL_SSP119_2030,TRL_SSP585_2040,TRL_SSP119_2040,TRL_SSP585_2050,TRL_SSP119_2050
0,V1A,dummy_tech,kilogram,9,9,9,9,9,9
1,V1B,LIB_NMC622,kWh,9,9,9,9,9,9
2,V1B,LIB_NCA,kWh,9,9,9,9,9,9
3,V1B,LIB_LFP,kWh,9,9,9,9,9,9
4,V1B,LIB_LTO,kWh,9,9,9,9,9,9


In [3]:
t2030_ssp119 = partial_TRL["TRL_SSP119_2030"].values
t2030_ssp585 = partial_TRL["TRL_SSP585_2030"].values
t2040_ssp119 = partial_TRL["TRL_SSP119_2040"].values
t2040_ssp585 = partial_TRL["TRL_SSP585_2040"].values
t2050_ssp119 = partial_TRL["TRL_SSP119_2050"].values
t2050_ssp585 = partial_TRL["TRL_SSP585_2050"].values

#### 1. linear interpolate between different SSP for a same year

In [4]:
def between_ssp_int(value1, value2, ratio1):
    """
    Linearly interpolate between two values. final TRL is round_down to int.
    value1, value2: use the available neighboring years/ssp for interpolating. All input / output single float
    ratio1: is the ratio to be multipled to value1, e.g., interp. ssp2 per ssp1 and ssp5, it should have a higher ratio e.g., 0.7 to ssp1
    """
    if value1 == 9:
        value = value1
    elif value2 == 9: 
        value = value2 
    else: 
        value = math.ceil(value1 * ratio1 + value2 * (1 - ratio1)) #same as math.floor( )  let's round_up, otherwise you won't have new tech for many Y[t] SSP[x] 
    return value

In [5]:
t2030_ssp126, t2030_ssp226, t2030_ssp245 = [], [], []

for x, y in zip(t2030_ssp119, t2030_ssp585):
    #print(x, y,  between_ssp_int (x,y, 0.8) ) 
    t2030_ssp126.append(between_ssp_int(x,y, 0.9))

for x, y in zip(t2030_ssp126, t2030_ssp585):
    #print(x, y,  between_ssp_int (x,y, 0.8) ) 
    t2030_ssp226.append(between_ssp_int(x,y, 0.7))

for x, y in zip(t2030_ssp226, t2030_ssp585):
    #print(x, y,  between_ssp_int (x,y, 0.8) ) 
    t2030_ssp245.append(between_ssp_int(x,y, 0.8))


In [6]:
t2040_ssp126, t2040_ssp226, t2040_ssp245 = [], [], []

for x, y in zip(t2040_ssp119, t2040_ssp585):
    #print(x, y,  between_ssp_int (x,y, 0.8) ) 
    t2040_ssp126.append(between_ssp_int(x,y, 0.9))

for x, y in zip(t2040_ssp126, t2040_ssp585):
    #print(x, y,  between_ssp_int (x,y, 0.8) ) 
    t2040_ssp226.append(between_ssp_int(x,y, 0.7))

for x, y in zip(t2040_ssp226, t2040_ssp585):
    #print(x, y,  between_ssp_int (x,y, 0.8) ) 
    t2040_ssp245.append(between_ssp_int(x,y, 0.8))


In [7]:
t2050_ssp126, t2050_ssp226, t2050_ssp245 = [], [], []

for x, y in zip(t2050_ssp119, t2050_ssp585):
    #print(x, y,  between_ssp_int (x,y, 0.8) ) 
    t2050_ssp126.append(between_ssp_int(x,y, 0.9))

for x, y in zip(t2050_ssp126, t2050_ssp585):
    #print(x, y,  between_ssp_int (x,y, 0.8) ) 
    t2050_ssp226.append(between_ssp_int(x,y, 0.7))

for x, y in zip(t2050_ssp226, t2050_ssp585):
    #print(x, y,  between_ssp_int (x,y, 0.8) ) 
    t2050_ssp245.append(between_ssp_int(x,y, 0.8))


#### 2. linear interpolate between different years for same SSP if more close v_year gap (by 5 years) needed

In [8]:
def between_yr_int(list1, list2):
    """
    Linearly interpolate between years once SSPx prepared, ratio = 0.5 for between two decades:
    """
    value = []
    for x, y in zip (list1, list2):
        value.append(x* 0.5+y*0.5)
    return value

In [9]:
t2035_ssp119 = between_yr_int(t2030_ssp119, t2040_ssp119)
t2035_ssp126 = between_yr_int(t2030_ssp126, t2040_ssp126)
t2035_ssp226 = between_yr_int(t2030_ssp226, t2040_ssp226) 
t2035_ssp245 = between_yr_int(t2030_ssp245, t2040_ssp245)
t2035_ssp585 = between_yr_int(t2030_ssp585, t2040_ssp585)

In [10]:
t2045_ssp119 = between_yr_int(t2040_ssp119, t2050_ssp119)
t2045_ssp126 = between_yr_int(t2040_ssp126, t2050_ssp126)
t2045_ssp226 = between_yr_int(t2040_ssp226, t2050_ssp226) 
t2045_ssp245 = between_yr_int(t2040_ssp245, t2050_ssp245)
t2045_ssp585 = between_yr_int(t2040_ssp585, t2050_ssp585)

#### 3. add all TRL back to DF

In [11]:
partial_TRL["TRL_SSP245_2030"] = t2030_ssp245
partial_TRL["TRL_SSP226_2030"] = t2030_ssp226
partial_TRL["TRL_SSP126_2030"] = t2030_ssp126 

partial_TRL["TRL_SSP245_2040"] = t2040_ssp245
partial_TRL["TRL_SSP226_2040"] = t2040_ssp226
partial_TRL["TRL_SSP126_2040"] = t2040_ssp126 

partial_TRL["TRL_SSP245_2050"] = t2050_ssp245
partial_TRL["TRL_SSP226_2050"] = t2050_ssp226
partial_TRL["TRL_SSP126_2050"] = t2050_ssp126 

In [12]:
# Order columns by column names
columns_to_order = partial_TRL.columns[2: ]
print(columns_to_order)

# Sort the column names
sorted_columns = sorted(columns_to_order)

# Reorder columns in DataFrame
TRL_finaldf = partial_TRL.reindex(columns=list(partial_TRL.columns[:2]) + sorted_columns)

Index(['LCIA_perunit', 'TRL_SSP585_2030', 'TRL_SSP119_2030', 'TRL_SSP585_2040',
       'TRL_SSP119_2040', 'TRL_SSP585_2050', 'TRL_SSP119_2050',
       'TRL_SSP245_2030', 'TRL_SSP226_2030', 'TRL_SSP126_2030',
       'TRL_SSP245_2040', 'TRL_SSP226_2040', 'TRL_SSP126_2040',
       'TRL_SSP245_2050', 'TRL_SSP226_2050', 'TRL_SSP126_2050'],
      dtype='object')


In [13]:
TRL_finaldf.head()  

,Module,Tech,LCIA_perunit,TRL_SSP119_2030,TRL_SSP119_2040,TRL_SSP119_2050,TRL_SSP126_2030,TRL_SSP126_2040,TRL_SSP126_2050,TRL_SSP226_2030,TRL_SSP226_2040,TRL_SSP226_2050,TRL_SSP245_2030,TRL_SSP245_2040,TRL_SSP245_2050,TRL_SSP585_2030,TRL_SSP585_2040,TRL_SSP585_2050
0,V1A,dummy_tech,kilogram,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9
1,V1B,LIB_NMC622,kWh,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9
2,V1B,LIB_NCA,kWh,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9
3,V1B,LIB_LFP,kWh,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9
4,V1B,LIB_LTO,kWh,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9


### 3. read in the "tech_toaddTRL.xlsx" prepared in notebook TRL_1A and addig TRL col

In [14]:
TRL_toadd = pd.read_excel("../data/to_use_interm/TRL_1A_tech_toaddTRL_wo_merged_header.xlsx") #, header = 1 for merged_cell version, index_col=[0,1,2,3,4]) 
TRL_toadd

,Unnamed: 0,v_type,size,powertrain,v_year,SSP,Module,Tech
0,0,garbage_truck,HDV_class7,ICEV-d,2030,ssp119,V1A,dummy_tech
1,1,garbage_truck,HDV_class7,ICEV-d,2030,ssp119,V1B,LIB_NMC622
2,2,garbage_truck,HDV_class7,ICEV-d,2030,ssp119,V1B,LIB_NCA
3,3,garbage_truck,HDV_class7,ICEV-d,2030,ssp119,V1B,LIB_LFP
4,4,garbage_truck,HDV_class7,ICEV-d,2030,ssp119,V1B,LIB_LTO
...,...,...,...,...,...,...,...,...
235,235,garbage_truck,HDV_class7,BEV,2050,ssp585,V1B,LSB
236,236,garbage_truck,HDV_class7,BEV,2050,ssp585,F2_fuel_diesel,diesel_mix
237,237,garbage_truck,HDV_class7,BEV,2050,ssp585,F2_fuel_elec,REC_100PV
238,238,garbage_truck,HDV_class7,BEV,2050,ssp585,F2_fuel_elec,grid_elec


In [15]:
#TRL_toadd[TRL_toadd['SSP'] != 'ssp370']
#TRL_toadd2 = TRL_toadd[TRL_toadd['SSP'] != 'ssp370']
#TRL_toadd2

In [16]:
trl_list = []
for index, row in TRL_toadd.iterrows():
    yr_value = row['v_year']
    ssp_value = row['SSP']
    tech = row['Tech']
    search_col  = "TRL_" + str(ssp_value).upper() + "_" + str(yr_value)
    if len(TRL_finaldf[TRL_finaldf["Tech"] == tech][search_col]) == 1: 
        value = TRL_finaldf[TRL_finaldf["Tech"] == tech][search_col].values[0]
    else:  ### for those with duplicate names e.g., dummy_tech, we always need it so TRL always 9
        value = 9 
    trl_list.append(value)

print(len(trl_list)) 

240


In [17]:
len(trl_list) == len(TRL_toadd)

True

In [18]:
TRL_toadd["TRL"] = trl_list
TRL_toadd

,Unnamed: 0,v_type,size,powertrain,v_year,SSP,Module,Tech,TRL
0,0,garbage_truck,HDV_class7,ICEV-d,2030,ssp119,V1A,dummy_tech,9
1,1,garbage_truck,HDV_class7,ICEV-d,2030,ssp119,V1B,LIB_NMC622,9
2,2,garbage_truck,HDV_class7,ICEV-d,2030,ssp119,V1B,LIB_NCA,9
3,3,garbage_truck,HDV_class7,ICEV-d,2030,ssp119,V1B,LIB_LFP,9
4,4,garbage_truck,HDV_class7,ICEV-d,2030,ssp119,V1B,LIB_LTO,9
...,...,...,...,...,...,...,...,...,...
235,235,garbage_truck,HDV_class7,BEV,2050,ssp585,V1B,LSB,5
236,236,garbage_truck,HDV_class7,BEV,2050,ssp585,F2_fuel_diesel,diesel_mix,9
237,237,garbage_truck,HDV_class7,BEV,2050,ssp585,F2_fuel_elec,REC_100PV,7
238,238,garbage_truck,HDV_class7,BEV,2050,ssp585,F2_fuel_elec,grid_elec,9


In [19]:
TRL_final = TRL_toadd.drop(TRL_toadd.columns[0], axis=1)
TRL_final

,v_type,size,powertrain,v_year,SSP,Module,Tech,TRL
0,garbage_truck,HDV_class7,ICEV-d,2030,ssp119,V1A,dummy_tech,9
1,garbage_truck,HDV_class7,ICEV-d,2030,ssp119,V1B,LIB_NMC622,9
2,garbage_truck,HDV_class7,ICEV-d,2030,ssp119,V1B,LIB_NCA,9
3,garbage_truck,HDV_class7,ICEV-d,2030,ssp119,V1B,LIB_LFP,9
4,garbage_truck,HDV_class7,ICEV-d,2030,ssp119,V1B,LIB_LTO,9
...,...,...,...,...,...,...,...,...
235,garbage_truck,HDV_class7,BEV,2050,ssp585,V1B,LSB,5
236,garbage_truck,HDV_class7,BEV,2050,ssp585,F2_fuel_diesel,diesel_mix,9
237,garbage_truck,HDV_class7,BEV,2050,ssp585,F2_fuel_elec,REC_100PV,7
238,garbage_truck,HDV_class7,BEV,2050,ssp585,F2_fuel_elec,grid_elec,9


In [20]:
# we'll do a further TRL table visualization in notebook 1C
TRL_final.to_excel("../data/to_use_interm/TRL_1B_tech_whTRLadded.xlsx", index= False)